In [1]:
def inf(text):
    print(f'INFO : {text}')
    
def err(text):
    print(f'ERROR: {text}')

In [2]:
# Imports and setup
from datetime import datetime, time
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
inf('All dependencies were imported successfully')

INFO : All dependencies were imported successfully


In [3]:
# Get config variable from environment variables
consumer_key = os.environ.get('twitter_sent_bot_key')
consumer_secret = os.environ.get('twitter_sent_bot_secret')
access_token = os.environ.get('twitter_sent_bot_token')
access_token_secret = os.environ.get('twitter_sent_bot_token_secret')
if None in [consumer_key, consumer_secret, access_token, access_token_secret]:
    err('Twitter auth tokens are not set as enviroment variables')
    exit(1)
else:
    inf('Twitter auth tokens are configured')

INFO : Twitter auth tokens are configured


In [4]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
inf('Twitter authentication is performed successfully')

INFO : Twitter authentication is performed successfully


In [5]:
self_user = api.me()
self_username = self_user['screen_name']
inf(f'The application is ran under {self_username} Twitter user')

INFO : The application is ran under ChaplyginAndrei Twitter user


In [6]:
def get_mentions(api, self_username, since_id=None):
    mention_count = 20
    if since_id:
        inf(f'Retrieving last {mention_count} mentions of user "{self_username}" done after {since_id}...')
    else:
        inf(f'Retrieving last {mention_count} mentions of user "{self_username}"')
    mentions = api.search(f'@{self_username}',rpp=mention_count, since_id=since_id).get('statuses')
    mentions = [{
        'id': mention['id'],
        'text': mention['text'],
        'user_mentions': [user_mention['screen_name'] for user_mention in mention['entities']['user_mentions']],
        'user' : mention['user']['screen_name']        
    } for mention in mentions]
    if len(mentions) == 0:
        inf(f'No mentions of "{self_username}" were retrieved at this time')
    else:
        inf(f'Retrieved {len(mentions)} mentions of user "{self_username}"')
    return mentions    

In [7]:
def get_tweets(api, target_username):
    #TODO: add logging
    max_id = None
    all_tweets = []
    pages = 1
    tweets_per_page = 100
    for page in range(1, pages + 1):
        tweets = api.user_timeline(target_username, page=page, count=tweets_per_page, max_id=max_id)
        tweets = [ {
            'id': tweet['id'],
            'text': tweet['text'],
        } for tweet in tweets]
        all_tweets += tweets
        if not max_id and len(tweets) > 0:
            max_id = tweets[0]['id']
        if len(tweets) < tweets_per_page:
            break
    return all_tweets 

In [8]:
def analyze_tweets(api, target_username):
    #TODO: check that analysis was not yet performed
    #TODO: add logging
    tweets = get_tweets(api, target_username)
    analyzer = SentimentIntensityAnalyzer()
    scores = [analyzer.polarity_scores(tweet['text'])['compound'] for tweet in tweets]
    return pd.DataFrame(scores, columns=['Polarity'], index=[-i for i in range(0, len(tweets))])

In [9]:
def get_color(polarity):
    return (0, polarity, 0) if polarity >= 0 else (-polarity, 0, 0)

In [10]:
def plot_analysis(data, target_username):
    plt.figure(figsize=(16, 8))
    avg_polarity = data['Polarity'].mean()
    avg_polarity_text = 'Very Positive' if avg_polarity >= 0.75 else \
                        'Positive' if avg_polarity >= 0.5 else \
                        'Positively Neutral' if avg_polarity >= 0 else \
                        'Negatively Neutral' if avg_polarity >= -0.5 else \
                        'Negative' if avg_polarity >= -0.75 else \
                        'Very Negative'
    color = get_color(avg_polarity)
    plt.plot(data.index, data['Polarity'], c=color, linewidth=1, linestyle='solid', marker='o', markersize=3)
    plt.xlim(-len(data.index) + 1, 0)
    plt.xlabel('Tweets Age', fontsize=16)
    plt.ylabel('Tweet Polarity', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks([-1, -0.5, 0, 0.5, 1], ['Strongly\nNegative', 'Likely\nNegative', 'Neutral', 'Likely\nPositive', 'Strongly\nPositive'], fontsize=16, ma='left')
    plt.title(f'Sentiment Analysis of @{target_username} Tweets ({datetime.strftime(datetime.now(), "%M/%D/%Y")}\n(Generally, a {avg_polarity_text} User)',fontsize=16)
    plt.grid(alpha=0.3)
    file_name = f'{target_username}.png'
    plt.savefig(file_name)
    return file_name

In [18]:
def post_analysis(api, file_name, target_username, in_reply_to_username, in_reply_to_status_id):
    tweet_text = f'Hi @{in_reply_to_username} here is the analysis of @{target_username} tweets'
    try:        
        api.update_with_media(file_name, tweet_text, in_reply_to_status_id=in_reply_to_status_id)
        inf(f'Successfully posted a tweet "{tweet_text}"')
    except Exception as e:
        err(f'Failed to post the following tweet "{tweet_text}"')
        err(e)

In [19]:
def post_already_analyzed_notifiction(api, target_user, reply_to_user, in_reply_to_status_id):
    tweet_text = f'Hi @{reply_to_user}, I\'ve already analyzed @{target_user} tweets, find it in my timeline'
    try:
        api.update_status(tweet_text, in_reply_to_status_id)
        inf(f'Successfully posted a tweet "{tweet_text}"')
    except Exception as e:
        err(f'Failed to post the following tweet "{tweet_text}"')
        err(e)    

In [12]:
analyzed_users = set()
notified_users = {}

In [ ]:
since_id = None
while True:
    mentions = get_mentions(api, self_username, since_id)
    if len(mentions) > 0:
        since_id = mentions[0]['id']
    for mention in mentions:
        tweet_text = mention['text']
        requested_by = mention['user']
        self_tweet = requested_by == self_username
        if self_tweet:
            inf(f'Tweet "{tweet_text}" is done by the target user himself, so no need to react to it')
            #continue
        all_mentions = mention['user_mentions']
        other_users_mentions = set(all_mentions)  
        other_users_mentions.discard(self_username)
        self_analysis = len(all_mentions) > 1 and len(other_users_mentions) == 0
        if self_analysis:
            inf(f'Tweet "{tweet_text}" is done by "{requested_by}" but asks "{self_username}" to analyze himself')
            #TODO: tweet about it
            continue
        if len(other_users_mentions) > 1:
            inf(f'Tweet "{tweet_text}" is done by "{requested_by}" but asks "{self_username}" to analyze multiple user')
            #TODO: tweet about it
            continue
        target_username = other_users_mentions.pop()
        # First we check if we've already analyzed the target user
        if target_username in analyzed_users:
            # If we did, then we check if we've already notified the requestor about it
            notified_requestor_about = notified_users.setdefault(requested_by, set())
            # If we did then we do nothing (so we don't duplicate the notification)
            if target_username in notified_requestor_about:
                continue
            else:
                notified_requestor_about.add(target_username)
                post_already_analyzed_notifiction(api, target_username, requested_by, mention['id'])
                continue
        else:
            analyzed_users.add(target_username)                
        # Peform the analysis
        tweets_analysis = analyze_tweets(api, target_username)
        plot_image = plot_analysis(tweets_analysis)
        post_analysis(api, plot_image, target_username, requested_by, mentiond['id'])        